In [10]:
from games_setup import *
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
import SBMLLint.common.constants as cn
import numpy as np
import pandas as pd
import scipy

In [35]:
simple = load_file_from_curated_data(40)
for r in simple.reactions:
  if r.category != cn.REACTION_BOUNDARY:
    print(r.makeIdentifier(is_include_kinetics=False))

Reaction1: Br + BrO3 -> HBrO2 + HOBr
Reaction2: Br + HBrO2 -> 2.00 HOBr
Reaction3: BrO3 + HBrO2 -> Ce + 2.00 HBrO2
Reaction4: 2.00 HBrO2 -> BrO3 + HOBr
Reaction5: Ce -> Br


In [36]:
mat_consistent = StoichiometryMatrix(simple)

In [37]:
mat_consistent.isConsistent()

True

In [92]:
mat_t = mat_consistent.stoichiometry_matrix.T
idx_mat_t = mat_t.index
# LU decomposition
mat_lu = scipy.linalg.lu(mat_t)
print(idx_mat_t)
print(type(idx_mat_t))
type(mat_lu)

Index(['Reaction1', 'Reaction2', 'Reaction3', 'Reaction4', 'Reaction5'], dtype='object')
<class 'pandas.core.indexes.base.Index'>


tuple

In [94]:
mat_t.columns

Index(['Ce', 'BrO3', 'Br', 'HOBr', 'HBrO2'], dtype='object')

In [93]:
mat_t

,Ce,BrO3,Br,HOBr,HBrO2
Reaction1,0.0,-1.0,-1.0,1.0,1.0
Reaction2,0.0,0.0,-1.0,2.0,-1.0
Reaction3,1.0,-1.0,0.0,0.0,1.0
Reaction4,0.0,1.0,0.0,1.0,-2.0
Reaction5,-1.0,0.0,1.0,0.0,0.0


In [60]:
p_inv = scipy.linalg.inv(mat_lu[0])
p_inv

array([[ 0., -0.,  1.,  0.,  0.],
       [ 1., -0.,  0.,  0.,  0.],
       [ 0., -0.,  0.,  0.,  1.],
       [ 0., -0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.]])

In [85]:
pivot_index = [list(k).index(1) for k in p_inv]
pivot_index

[2, 0, 4, 3, 1]

In [87]:
new_idx_mat_t = [idx_mat_t[idx] for idx in pivot_index]
new_idx_mat_t

['Reaction3', 'Reaction1', 'Reaction5', 'Reaction4', 'Reaction2']

In [78]:
[k for k in p_inv]

[array([ 0., -0.,  1.,  0.,  0.]),
 array([ 1., -0.,  0.,  0.,  0.]),
 array([ 0., -0.,  0.,  0.,  1.]),
 array([ 0., -0.,  0.,  1.,  0.]),
 array([0., 1., 0., 0., 0.])]

In [58]:
np.matmul(scipy.linalg.inv(mat_lu[0]), pd.DataFrame(idx_mat_t).index)

array([2., 0., 4., 3., 1.])

In [42]:
mat_lu[1]

array([[ 1. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  1. ,  0. ,  0. ,  0. ],
       [-1. ,  1. ,  1. ,  0. ,  0. ],
       [ 0. , -1. , -0.5,  1. ,  0. ],
       [ 0. , -0. , -0.5,  1. ,  1. ]])

In [43]:
for r in simple.reactions:
  if r.category != cn.REACTION_BOUNDARY:
    print(r.makeIdentifier(is_include_kinetics=False))

Reaction1: Br + BrO3 -> HBrO2 + HOBr
Reaction2: Br + HBrO2 -> 2.00 HOBr
Reaction3: BrO3 + HBrO2 -> Ce + 2.00 HBrO2
Reaction4: 2.00 HBrO2 -> BrO3 + HOBr
Reaction5: Ce -> Br


In [44]:
mat_lu[2]

array([[ 1. , -1. ,  0. ,  0. ,  1. ],
       [ 0. , -1. , -1. ,  1. ,  1. ],
       [ 0. ,  0. ,  2. , -1. ,  0. ],
       [ 0. ,  0. ,  0. ,  1.5, -1. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ]])

In [88]:
pivot_mat = np.matmul(scipy.linalg.inv(mat_lu[0]), mat_t)
pivot_mat

array([[ 1., -1.,  0.,  0.,  1.],
       [ 0., -1., -1.,  1.,  1.],
       [-1.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  1., -2.],
       [ 0.,  0., -1.,  2., -1.]])

In [95]:
reshuffled_mat = pd.DataFrame(pivot_mat, index=new_idx_mat_t, columns=mat_t.columns)
reshuffled_mat

,Ce,BrO3,Br,HOBr,HBrO2
Reaction3,1.0,-1.0,0.0,0.0,1.0
Reaction1,0.0,-1.0,-1.0,1.0,1.0
Reaction5,-1.0,0.0,1.0,0.0,0.0
Reaction4,0.0,1.0,0.0,1.0,-2.0
Reaction2,0.0,0.0,-1.0,2.0,-1.0


In [48]:
# operation matrix (L^-1)
scipy.linalg.inv(mat_lu[1])

array([[ 1. ,  0. ,  0. ,  0. , -0. ],
       [ 0. ,  1. ,  0. ,  0. , -0. ],
       [ 1. , -1. ,  1. ,  0. , -0. ],
       [ 0.5,  0.5,  0.5,  1. , -0. ],
       [ 0. , -1. ,  0. , -1. ,  1. ]])

In [97]:
reduced_mat = pd.DataFrame(mat_lu[2], index=new_idx_mat_t, columns=mat_t.columns)
reduced_mat

,Ce,BrO3,Br,HOBr,HBrO2
Reaction3,1.0,-1.0,0.0,0.0,1.0
Reaction1,0.0,-1.0,-1.0,1.0,1.0
Reaction5,0.0,0.0,2.0,-1.0,0.0
Reaction4,0.0,0.0,0.0,1.5,-1.0
Reaction2,0.0,0.0,0.0,0.0,0.0


In [34]:
mat_consistent.stoichiometry_matrix

,Mdm2Synthesis,Mdm2mRNASynthesis,Mdm2mRNADegradation,Mdm2Degradation,p53Synthesis,p53Degradation,P53_Mdm2Binding,P53_Mdm2Release,DNArepair,ARFactivation,ARF_Mdm2Binding,ARF_Mdm2Degradation,ARFDegradation
p53syn,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ARF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,1.0,-1.0
Mdm2mRNAsyn,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mdm2deg,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Sink,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
Mdm2mRNAdeg,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p53,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0
Mdm2,1.0,0.0,0.0,-1.0,0.0,1.0,-1.0,1.0,0.0,0.0,-1.0,0.0,0.0
damDNA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0
mdm2syn,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
mat_consistent.stoichiometry_matrix.shape[1]

4

In [35]:
mat_consistent.isConsistent()

False

In [7]:
simple_inconsistent.reactions

[reaction1:  -> C; vi,
 reaction2: C -> ; C * k1 * X * pow(C + K5, -1),
 reaction3: C -> ; C * kd,
 reaction4:  -> M; (1 + -1 * M) * V1 * pow(K1 + -1 * M + 1, -1),
 reaction5: M -> ; M * V2 * pow(K2 + M, -1),
 reaction6:  -> X; V3 * (1 + -1 * X) * pow(K3 + -1 * X + 1, -1),
 reaction7: X -> ; V4 * X * pow(K4 + X, -1),
 reaction8: C + Y -> Z; a1 * C * Y,
 reaction9: Z -> C + Y; a2 * Z,
 reaction10: Z -> C; alpha * d1 * Z,
 reaction11: Z -> Y; alpha * kd * Z,
 reaction12:  -> Y; vs,
 reaction13: Y -> ; d1 * Y]

In [10]:
mat_consistent.stoichiometry_matrix.shape

(4, 4)

In [12]:
mat_inconsistent = StoichiometryMatrix(simple_inconsistent)
mat_inconsistent.stoichiometry_matrix

,reaction8,reaction9,reaction10,reaction11
Z,1.0,-1.0,-1.0,-1.0
C,-1.0,1.0,1.0,0.0
Y,-1.0,1.0,0.0,1.0


In [13]:
mat_inconsistent.stoichiometry_matrix.shape

(3, 4)

In [64]:
print(mat_consis.isConsistent())

False


In [18]:
reaction_labels = [r.label for r in simple.reactions]
molecule_names = [m.name for m in simple.molecules]
stoichiometry_matrix = pd.DataFrame(0.0, index=molecule_names, columns=reaction_labels)
stoichiometry_matrix['Reaction1']['Br'] = 3.8
stoichiometry_matrix

KeyError: 'Reaction1'

In [16]:
reactants = {r.molecule.name:r.stoichiometry for r in reaction.reactants}
products = {p.molecule.name:p.stoichiometry for p in reaction.products}
print(reactants)
print(type(reactants))
print(products)

{'BrO3': 1.0, 'HBrO2': 1.0}
<class 'dict'>
{'Ce': 1.0, 'HBrO2': 2.0}


In [32]:
set(reactants.keys()).union(products.keys())

{'BrO3', 'Ce', 'HBrO2'}

In [13]:
sub_reactants = {r for r in reactants if simple.getMolecule(r.name)}
sub_products = {r for r in products if simple.getMolecule(r.name)}
print(sub_reactants)

{HBrO2, BrO3}


In [11]:
reactants.intersection(simple.molecules)

set()

In [12]:
products.intersection(simple.molecules)

{Ce}